# Expérience 1 

## Entrainez un classificateur sur MNIST (Resnet18 poids initiaux aléatoire)

### Imports

In [1]:
import argparse, os, random, csv, time 
import numpy as np
from pathlib import Path 

import torch 
import torch.nn as nn 
import torch.nn.init as init
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models 
from tqdm import tqdm
from torchvision.transforms import InterpolationMode


### **Obtention des données de standardisations** :

Pour les images du données de jeu de ImageNet

In [2]:
from torchvision.models import ResNet18_Weights
print(ResNet18_Weights.IMAGENET1K_V1.transforms().mean)
print(ResNet18_Weights.IMAGENET1K_V1.transforms().std)


[0.485, 0.456, 0.406]
[0.229, 0.224, 0.225]


Pour MNIST

In [5]:
from torchvision import datasets, transforms
import torch

train_set = datasets.MNIST("./data", train=True, download=True, transform=transforms.ToTensor())
data = torch.cat([x for x, _ in train_set], dim=0)  # concatène toutes les images
mean = data.mean().item()
std = data.std().item()
print("Moyenne : ",mean)
print("Std : ", std)


Moyenne :  0.13066047430038452
Std :  0.30810782313346863


In [6]:
IMAGENET_MEAN = (0.485, 0.456, 0.406)
IMAGENET_STD  = (0.229, 0.224, 0.225)
MNIST_MEAN = (0.1307, 0.1307, 0.1307)  # dupliqué sur 3 canaux
MNIST_STD  = (0.3081, 0.3081, 0.3081)

### Définir la seed + device

In [7]:
def set_seed(seed: int | None):
    """
    Fixe toutes les graines aléatoires pour garantir la reproductibilité des expériences.

    Cette fonction initialise les générateurs de nombres aléatoires utilisés par :
      - le module `random` (Python standard)
      - `numpy` (opérations et tirages aléatoires)
      - `torch` (initialisation des poids, dropout, DataLoader, etc.)
      - `torch.cuda` (opérations GPU)

    Elle rend également les opérations cuDNN déterministes pour assurer
    des résultats identiques sur GPU entre plusieurs exécutions.

    Paramètres
    ----------
    seed : int ou None
        - Si un entier est fourni : active le mode déterministe avec cette graine.
        - Si None : ne fait rien (l'entraînement reste aléatoire).
    """
    if seed is None:
        return
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

In [8]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
DEVICE

device(type='cpu')

### Transform Data

In [9]:
def build_transform(pretrained: bool = True, input_size: int = 32, center_crop: bool = False):
    """
    - input_size : taille finale (ex: 64 ou 96 ou 128)
    - center_crop=False : on évite un crop inutile pour MNIST; mets True si tu veux Resize+Crop
    - pretrained=True : normalisation ImageNet
      pretrained=False: normalisation MNIST (3 canaux)
    """
    ops = []
    if center_crop:
        # Variante "classique" Resize -> CenterCrop (un poil plus lent)
        ops += [
            transforms.Resize(input_size + input_size // 8, interpolation=InterpolationMode.BILINEAR, antialias=True),
            transforms.CenterCrop(input_size),
        ]
    else:
        # Plus rapide: un seul Resize direct à la bonne taille
        ops += [
            transforms.Resize(input_size, interpolation=InterpolationMode.BILINEAR, antialias=True),
        ]

    # MNIST est en niveaux de gris: on fait 3 canaux proprement
    ops += [
        transforms.Grayscale(num_output_channels=3),  # plus propre que lambda x.convert("RGB")
        transforms.ToTensor(),
    ]

    if pretrained:
        ops += [transforms.Normalize(IMAGENET_MEAN, IMAGENET_STD)]
    else:
        ops += [transforms.Normalize(MNIST_MEAN, MNIST_STD)]

    return transforms.Compose(ops)

### Model + Init weigths + choix optimiseur + choix crit (f° de loss)

In [10]:
# A compléter au fur et à mesure que l'on choisira comment initialiser nos poids

def init_weights(module: nn.Module, mode: str = "default"):
    if mode == "default":
        return


In [11]:
def build_resnet18(num_classes = 10, pretrained = False, init_mode = "default", freeze_backbone = False):
    weigths = models.ResNet18_Weights.IMAGENET1K_V1 if pretrained else None
    model = models.resnet18(weights = weigths)
    in_f = model.fc.in_features
    # On regarde à la fin le nb d'entrées attendues par la dernière couche du resnet
    # on s'en sert pour remplacer la derniere couche du resnet (1000 classe pour imaganet)
    # par la nouvelle couche adaptée à MNIST (donc 10 classes)

    model.fc = nn.Linear(in_f, num_classes)

    if freeze_backbone : 
        for name, p in model.named_parameters():
            if not name.startswith("fc."):
                p.requires_grad = False
        
    if init_mode == "head_only":
        init_weights(model.fc, mode = "default")
    elif init_mode in {}:
        model.apply(lambda m: init_weights(m, init_mode))
    return model 

In [12]:
def make_optimizer(name: str, params, lr: float, weight_decay: float = 0.0):
    """
    Crée un optimiseur à partir de son nom.
    Exemples  :
      - SGD
      - Adam/AdamW
      - RMSprop
      - rmsprop 
      - adagrad
    """
    n = name.lower()
    if n == "adamw":
        return optim.AdamW(params, lr=lr, weight_decay=weight_decay)
    if n == "adam":
        return optim.Adam(params, lr=lr, weight_decay=weight_decay)
    if n == "sgd":
        return optim.SGD(params, lr=lr, weight_decay=weight_decay)
    if n == "rmsprop":
        return optim.RMSprop(params, lr=lr, weight_decay=weight_decay)
    if n == "adagrad":
        return optim.Adagrad(params, lr=lr, weight_decay=weight_decay)
    raise ValueError(f"Optimiseur inconnu: {name}")

In [13]:
def make_criterion(name: str):
    """
    Crée une loss de classification à partir de son nom.
    Exempless:
      - CrossEntropyLoss
      - 
    """
    n = name.lower()
    if n in ("crossentropy", "crossentropyloss", "ce"):
        return nn.CrossEntropyLoss()
    raise ValueError(f"Loss inconnue: {name}")

In [14]:
def make_scheduler(name: str, optimizer, epochs: int):
    n = name.lower()

    if n == "step":
        return torch.optim.lr_scheduler.StepLR(optimizer, step_size=epochs // 3, gamma=0.1)
    # gamma : facteur de réduction 



    if n == "cosine":
        return torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=epochs)

    if n == "none":
        return None

    raise ValueError(f"Scheduler inconnu: {name}")


### Boucles entrainement / evaluation

In [15]:
def evaluate(model, loader, criterion): 
    model.eval()
    loss_sum =0.0; correct = 0; total = 0
    for x, y in loader :
        x,y = x.to(DEVICE), y.to(DEVICE)
        logits = model(x)
        loss = criterion(logits, y)
        loss_sum+=loss.item()*x.size(0)
        correct += (logits.argmax(1) == y).sum().item()
        total += x.size(0)
    return loss_sum/total, correct/total 

In [ ]:
def run_train(

    pretrained=False,             # True = poids ImageNet + normalisation ImageNet
    epochs=10,
    batch_size=128,
    lr=1e-3,
    weight_decay=1e-4,
    seed=42,                      # None pour laisser aléatoire (utile pour l'ensemble)
    data_dir="./data",
    out_dir="./runs",
    save_tag=None,
    # nouveau : init & freeze
    init_mode="default",          # 'default' pour l'instant que ça mais à améliorer pour offrir différentes initialisation de poids
    freeze_backbone=False,        # gèle le backbone (pré-entraîné) et n'entraîne que la tête
    shuffle = True ,
    optimiseur = "AdamW", 
    criterion = "CrossEntropyLoss",
    schedular = "none"              # activer un scheduler ou non : "step" | "cosine" | "none" | par défaut None
):

    set_seed(seed)


    # Partie Data 

    tf = build_transform(pretrained)
    train_set = datasets.MNIST(data_dir, train = True, download = True, transform = tf)
    test_set = datasets.MNIST(data_dir, train = False, download = False, transform =tf)
    train_loader = DataLoader(train_set, batch_size=batch_size, shuffle = shuffle, num_workers = 0 , pin_memory=False)
    test_loader  = DataLoader(test_set,  batch_size=256, shuffle=False, num_workers=0, pin_memory=False)




    # Partie model 
    model = build_resnet18(
        num_classes = 10,
        pretrained = pretrained,
        init_mode = init_mode, 
        freeze_backbone=freeze_backbone

    ).to(DEVICE)
    params = [p for p in model.parameters() if p.requires_grad]
    opt = make_optimizer(optimiseur, params, lr, weight_decay)
    crit = make_criterion(criterion)
    sched = make_scheduler(schedular, opt, epochs)



    # logs 
    ts = time.strftime("%Y%m%d-%H%M%S")
    tag = save_tag or f"resnet18_{'pre' if pretrained else 'scratch'}_{init_mode}{'_frozen' if freeze_backbone else ''}_{ts}"
    run_dir = Path(out_dir)/tag; run_dir.mkdir(parents=True, exist_ok=True)
    csv_path = run_dir/"metrics.csv"
    with open(csv_path,"w",newline="") as f: csv.writer(f).writerow(["epoch","train_loss","train_acc","val_loss","val_acc"])
    best_path = run_dir/"best.pt"; last_path = run_dir/"last.pt"
    best_acc = 0.0

    # Boucle d'entrainement 
    for ep in range(1, epochs+1):
        print("Début de l'epochs : ",ep)
        model.train(); run_loss =0.0; run_corr=0; run_tot = 0
        for x, y in train_loader : 
            x = x.to(DEVICE)
            y = y.to(DEVICE)
            opt.zero_grad()
            logits = model(x)
            loss = crit(logits, y)
            loss.backward()
            opt.step()
            
            run_loss += loss.item()*x.size(0)
            run_corr += (logits.argmax(1) == y).sum().item()
            run_tot += x.size(0)


   
        tl, ta = run_loss/run_tot, run_corr/run_tot
        vl, va = evaluate(model, test_loader, crit)

        if sched is not None :
            sched.step()
        with open(csv_path,"a",newline="") as f: csv.writer(f).writerow([ep,f"{tl:.6f}",f"{ta:.4f}",f"{vl:.6f}",f"{va:.4f}"])
        if va>best_acc:
            best_acc=va
            torch.save({"state_dict":model.state_dict(),
                        "pretrained":pretrained,
                        "init_mode":init_mode,
                        "freeze_backbone":freeze_backbone}, best_path)
        torch.save({"state_dict":model.state_dict(),
                    "pretrained":pretrained,
                    "init_mode":init_mode,
                    "freeze_backbone":freeze_backbone}, last_path)
        print(f"[{ep:02d}/{epochs}] train_loss={tl:.4f} acc={ta:.4f} | val_loss={vl:.4f} acc={va:.4f}")

    print(f"Best val acc={best_acc:.4f} | best ckpt={best_path}")
    return str(best_path), str(csv_path)
        


### Récupértion des données de MNIST

## Expérience 1 

In [70]:
ckpt, log = run_train(
    pretrained=True,
    freeze_backbone=True,
    init_mode="head_only",
    epochs=10,
    batch_size=64,          # 64 sur CPU est souvent plus fluide que 128
    lr=2e-3,
    weight_decay=1e-4,
    optimiseur="AdamW",
    criterion="CrossEntropyLoss",
    save_tag="fast_headonly_pretrained",
)


Début de l'epochs :  1
[01/10] train_loss=0.8793 acc=0.7210 | val_loss=0.7229 acc=0.7771
Début de l'epochs :  2
[02/10] train_loss=0.7613 acc=0.7582 | val_loss=0.6941 acc=0.7905
Début de l'epochs :  3
[03/10] train_loss=0.7496 acc=0.7623 | val_loss=0.7025 acc=0.7864
Début de l'epochs :  4
[04/10] train_loss=0.7436 acc=0.7663 | val_loss=0.7413 acc=0.7699
Début de l'epochs :  5
[05/10] train_loss=0.7373 acc=0.7676 | val_loss=0.6858 acc=0.7938
Début de l'epochs :  6
[06/10] train_loss=0.7369 acc=0.7677 | val_loss=0.6770 acc=0.7953
Début de l'epochs :  7
[07/10] train_loss=0.7376 acc=0.7688 | val_loss=0.7253 acc=0.7790
Début de l'epochs :  8
[08/10] train_loss=0.7284 acc=0.7709 | val_loss=0.6818 acc=0.7902
Début de l'epochs :  9
[09/10] train_loss=0.7354 acc=0.7678 | val_loss=0.6846 acc=0.7930
Début de l'epochs :  10
[10/10] train_loss=0.7331 acc=0.7691 | val_loss=0.7505 acc=0.7685
Best val acc=0.7953 | best ckpt=runs\fast_headonly_pretrained\best.pt
